In [33]:
# @title Environment Setup
import os
import sys

# 1. *** FIX: Clear problematic environment variable for matplotlib ***
# This prevents the "ValueError: Key backend: 'module://matplotlib_inline.backend_inline'" error
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
if not os.path.exists("font_diffusion"):
    print("⬇️ Cloning repository...")
    !git clone https://github.com/dzungphieuluuky/font_diffusion.git

# 3. Install PyTorch 1.13
print("\n⬇️ Installing PyTorch 1.13 (Required for this model)...")
# Force reinstall torch 1.13 to match the model's training environment
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!pip install xformers==0.0.16 -q

# Install Transformers & Diffusers
!pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
!pip install lpips scikit-image
# -----------------------------------------------------------------
!sudo apt-get update && sudo apt-get install dos2unix
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

⬇️ Cloning repository...
Cloning into 'font_diffusion'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 151 (delta 34), reused 132 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (151/151), 13.71 MiB | 17.55 MiB/s, done.
Resolving deltas: 100% (34/34), done.

⬇️ Installing PyTorch 1.13 (Required for this model)...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
ERROR: Could not find a version that satisfies the requirement torch==1.13.1+cu117 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==1.13.1+cu117

⬇️ Installing Dependencies (Manually fixed)...
  Using cached transformers-4.33.1-py3-none-any.whl.metadata (119 kB)
  Using cached accelerate-0.23.0-py3-none-any.whl.metadata (18 kB)
  Using cached diffusers-0.22.0

In [34]:
# @title Environment Inspection
import os
import sys
import platform
import subprocess

# --- Environment Detection and Path Setup ---

def detect_env_and_setup_paths():
    """
    Detects the current environment (Kaggle, Colab, or Local) and
    returns appropriate paths and environment name.
    """
    env_name = "Local/Other"
    input_dir = "input/"
    output_dir = "output/"

    # 1. Detect Kaggle
    if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        env_name = "Kaggle"
        # Kaggle's standard paths
        input_dir = "/kaggle/input/"
        output_dir = "/kaggle/working/"

    # 2. Detect Colab (Note: Colab also has 'COLAB_GPU' if a GPU is assigned)
    elif 'google.colab' in sys.modules:
        env_name = "Colab"
        # Colab's default paths relative to the content folder
        input_dir = "/content/" # Often you mount Drive or download here
        output_dir = "/content/" # Default working directory
        print("Note: In Colab, you might need to mount Google Drive manually for persistent storage:")
        print("      from google.colab import drive; drive.mount('/content/drive')")

    # 3. Default to Local/Other
    # Paths are set to a simple structure relative to the script location

    return env_name, input_dir, output_dir

# --- System and Package Info ---

def get_system_info(env_name):
    """Prints Python, PyTorch, CUDA, and GPU information."""
    print("\n" + "="*50)
    print(f"       💻 System and Package Information for {env_name}")
    print("="*50)

    # 1. Python Version
    print(f"**Python Version:** {sys.version.split()[0]} ({platform.python_implementation()})")

    # 2. PyTorch and CUDA Info
    try:
        import torch
        print(f"**PyTorch Version:** {torch.__version__}")

        if torch.cuda.is_available():
            print("\n**CUDA/GPU Information (PyTorch):**")
            # CUDA version
            print(f"  - CUDA is Available: **True**")
            print(f"  - CUDA Version (Runtime): {torch.version.cuda}")
            # GPU details
            gpu_count = torch.cuda.device_count()
            print(f"  - GPU Count: {gpu_count}")
            for i in range(gpu_count):
                print(f"  - Device {i}: {torch.cuda.get_device_name(i)}")
        else:
            print(f"  - CUDA is Available: **False** (Running on CPU)")
    except ImportError:
        print("\n**PyTorch:** Not installed or not found.")
    except Exception as e:
        print(f"\n**PyTorch/CUDA Check Error:** {e}")

    # 3. nvidia-smi (System-level GPU info)
    print("\n**NVIDIA-SMI Output (Raw Driver/System Info):**")
    try:
        # Run nvidia-smi command
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
        print(result.stdout)
    except FileNotFoundError:
        # This will happen if nvidia-smi is not in PATH or no NVIDIA driver is installed
        print("  - `nvidia-smi` command not found (No NVIDIA GPU/drivers or not in PATH).")
    except subprocess.CalledProcessError as e:
        # This might happen if the command runs but fails (e.g., driver issues)
        print(f"  - `nvidia-smi` failed to run. Error: {e.stderr.strip()}")

# --- Main Execution ---

if __name__ == "__main__":

    # Get environment details and paths
    env_name, INPUT_DIR, OUTPUT_DIR = detect_env_and_setup_paths()

    # Print detected environment and paths
    print("="*50)
    print(f"       ✅ Environment Detected: **{env_name}**")
    print("="*50)
    print(f"**Input Path (Default):** {INPUT_DIR}")
    print(f"**Output Path (Default):** {OUTPUT_DIR}")

    # Run system checks
    get_system_info(env_name)

    print("="*50)

# Example Usage within the script (simulated)
# train_data_path = os.path.join(INPUT_DIR, "dataset_folder", "train.csv")
# model_save_path = os.path.join(OUTPUT_DIR, "best_model.pth")

Note: In Colab, you might need to mount Google Drive manually for persistent storage:
      from google.colab import drive; drive.mount('/content/drive')
       ✅ Environment Detected: **Colab**
**Input Path (Default):** /content/
**Output Path (Default):** /content/

       💻 System and Package Information for Colab
**Python Version:** 3.12.12 (CPython)
**PyTorch Version:** 2.9.0+cu126

**CUDA/GPU Information (PyTorch):**
  - CUDA is Available: **True**
  - CUDA Version (Runtime): 12.6
  - GPU Count: 1
  - Device 0: Tesla T4

**NVIDIA-SMI Output (Raw Driver/System Info):**
Sat Dec 13 11:22:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf 

In [35]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_DIR, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_DIR}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -q -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

Unzipping /content/fonts.zip...
Unzipping of /content/fonts.zip complete.
Unzipping /content/ckpt.zip...
Unzipping of /content/ckpt.zip complete.
Unzipping /content/font_generations.zip...
Unzipping of /content/font_generations.zip complete.


In [36]:
# @title Checking checkpoint files (.pth)
import os
import time

CHECKPOINT_DIR = os.path.join(INPUT_DIR, "ckpt")
print(CHECKPOINT_DIR)
# Create the checkpoint directory
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
# Wait loop to check if files exist
required_files = ["unet.pth", "content_encoder.pth", "style_encoder.pth"]

while True:
    missing = [f for f in required_files if not os.path.exists(f"{CHECKPOINT_DIR}/{f}")]

    if not missing:
        print("\n✅ All weights found! You can proceed to the next step.")
        break
    else:
        print(f"Waiting for files... Missing: {missing}")
        print("Upload them to the 'ckpt' folder now.")
        time.sleep(10) # Checks every 10 seconds

/content/ckpt
⚠️ AUTOMATED DOWNLOAD FAILED (The author made the model private).
----------------------------------------------------------------
1. Please download 'unet.pth', 'content_encoder.pth', and 'style_encoder.pth'
   from the official GitHub Model Zoo: https://github.com/yeungchenwa/FontDiffuser
2. Look at the file sidebar on the left of this screen.
3. Open the 'FontDiffuser' folder, then open the 'ckpt' folder.
4. Drag and drop the 3 files into that 'ckpt' folder.
----------------------------------------------------------------

✅ All weights found! You can proceed to the next step.


In [37]:
%%writefile ablation.py
# @title Ablation Experiments
import os
import subprocess
import itertools
from pathlib import Path
import argparse
# --- 1. CONFIGURATION ---
# Setup argument parser
parser = argparse.ArgumentParser(description='Run an ablation experiment for FontDiffuser.')
parser.add_argument('--content_image_path', type=str, default='data_examples/content.png',
                    help='Path to the content image.')
parser.add_argument('--style_image_path', type=str, default='data_examples/style.png',
                    help='Path to the style image.')

args, unknown = parser.parse_known_args()

CONTENT_IMAGE_PATH = args.content_image_path
STYLE_IMAGE_PATH = args.style_image_path

# Output Directory
OUTPUT_DIR = "outputs_ablation"

# --- 2. DEFINE PARAMETERS TO TEST ---
# "Guidance Scale": Higher = Forces the style more strictly. Lower = More creative/random.
scales = [i + 0.5 for i in range(5, 10)]

# "Inference Steps": Higher = Cleaner, more detailed strokes. Lower = Faster, rougher.
steps = range(20, 151, 10)

# --- 3. SETUP ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
content_basename = Path(CONTENT_IMAGE_PATH).stem
style_basename = Path(STYLE_IMAGE_PATH).stem

# Generate all possible combinations
combinations = list(itertools.product(scales, steps))

print(f"🚀 Starting Ablation Study: {len(combinations)} experiments")
print(f"📂 Inputs: {content_basename} + {style_basename}")
print(f"📂 Output Directory: {OUTPUT_DIR}")

# --- 4. MAIN LOOP ---
for i, (scale, step) in enumerate(combinations):
    print(f"\n[{i+1}/{len(combinations)}] Testing: Scale={scale}, Steps={step}...")

    # Construct the command
    cmd = [
        "python", "font_diffusion/sample.py",
        "--ckpt_dir", "ckpt/",
        "--content_image_path", CONTENT_IMAGE_PATH,
        "--style_image_path", STYLE_IMAGE_PATH,
        "--save_image",
        "--save_image_dir", OUTPUT_DIR,
        "--device", "cuda:0",
        "--algorithm_type", "dpmsolver++",
        "--guidance_type", "classifier-free",
        "--method", "multistep",
        # Dynamic Parameters
        "--guidance_scale", str(scale),
        "--num_inference_steps", str(step)
    ]

    try:
        # Run the generation script
        subprocess.run(cmd, check=True)

        # --- 5. RENAME OUTPUTS ---
        # Define meaningful filenames
        # Format: result_scale-7.5_step-50.png
        filename_suffix = f"scale-{scale}_step-{step}"

        target_single = os.path.join(OUTPUT_DIR, f"result_{filename_suffix}.png")
        target_compare = os.path.join(OUTPUT_DIR, f"compare_{filename_suffix}.png")

        # Default outputs from sample.py
        src_single = os.path.join(OUTPUT_DIR, "out_single.png")
        src_compare = os.path.join(OUTPUT_DIR, "out_with_cs.png")

        # Rename "Single Result"
        if os.path.exists(src_single):
            os.rename(src_single, target_single)
            print(f"   ✅ Saved: {os.path.basename(target_single)}")
        else:
            print("   ⚠️ Warning: out_single.png not found.")

        # Rename "Comparison Grid" (Content | Style | Result)
        if os.path.exists(src_compare):
            os.rename(src_compare, target_compare)

    except subprocess.CalledProcessError:
        print(f"   ❌ Error running experiment {filename_suffix}")
    except Exception as e:
        print(f"   ⚠️ Unexpected error: {e}")

print("\n🎉 Ablation study complete!")
"""Example
!python ablation.py \
  --content_image_path /content/content.jpg \
  --style_image_path /content/sinonom_diffuser/figures/ref_imgs/ref_雕.jpg
"""

Overwriting ablation.py


In [38]:
%%writefile ablation_analysis.py
# @title Ablation Analysis
import os
import torch
import lpips
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import re

# --- CONFIG ---
results_folder = "outputs_ablation"
content_ref_path = "/content/content.jpg"
style_ref_path = "/content/font_diffusion/figures/ref_imgs/ref_媚.jpg"

# Load LPIPS Metric
loss_fn_alex = lpips.LPIPS(net='alex').cuda()

def load_tensor(path):
    img = cv2.imread(path)
    if img is None: return None
    img = cv2.resize(img, (128, 128))
    img = (img / 255.0) * 2 - 1
    img = np.transpose(img, (2, 0, 1))
    return torch.tensor(img, dtype=torch.float32).unsqueeze(0).cuda()

def calculate_ssim(img_path1, img_path2):
    i1 = cv2.imread(img_path1, cv2.IMREAD_GRAYSCALE)
    i2 = cv2.imread(img_path2, cv2.IMREAD_GRAYSCALE)
    if i1 is None or i2 is None: return 0.0
    i1 = cv2.resize(i1, (128, 128))
    i2 = cv2.resize(i2, (128, 128))
    return ssim(i1, i2)

# --- DATA COLLECTION ---
print(f"{'Filename':<45} | {'SSIM':<10} | {'LPIPS':<10}")
print("-" * 75)

tensor_style = load_tensor(style_ref_path)

# Dictionary to store data: data[scale] = {step: (ssim, lpips)}
plot_data = {}

files = sorted([f for f in os.listdir(results_folder) if f.endswith(".png") and "result_" in f])

for f in files:
    gen_path = os.path.join(results_folder, f)

    # 1. Calculate Metrics
    score_ssim = calculate_ssim(content_ref_path, gen_path)

    tensor_gen = load_tensor(gen_path)
    if tensor_gen is not None:
        with torch.no_grad():
            score_lpips = loss_fn_alex(tensor_gen, tensor_style).item()
    else:
        score_lpips = 1.0 # Max error if file fail

    print(f"{f:<45} | {score_ssim:.4f}     | {score_lpips:.4f}")

    # 2. Parse Filename to get Parameters
    # Expected format: result_scale-7.5_step-50.png
    try:
        # Regex to extract numbers from "scale-X.X" and "step-XX"
        scale_match = re.search(r"scale-([\d\.]+)", f)
        step_match = re.search(r"step-([\d]+)", f)

        if scale_match and step_match:
            scale = float(scale_match.group(1))
            step = int(step_match.group(1))

            # Store data
            if scale not in plot_data:
                plot_data[scale] = {'steps': [], 'ssim': [], 'lpips': []}

            plot_data[scale]['steps'].append(step)
            plot_data[scale]['ssim'].append(score_ssim)
            plot_data[scale]['lpips'].append(score_lpips)
    except Exception as e:
        print(f"⚠️ Could not parse parameters from {f}")

# --- VISUALIZATION ---
print("\n📊 Generating Graphs...")

# Make figure slightly wider to accommodate the external legend
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

sorted_scales = sorted(plot_data.keys())

for scale in sorted_scales:
    data = plot_data[scale]
    sorted_points = sorted(zip(data['steps'], data['ssim'], data['lpips']))
    steps, ssims, lpip_scores = zip(*sorted_points)

    # Plot Lines
    ax1.plot(steps, ssims, marker='o', label=f"Guidance {scale}")
    ax2.plot(steps, lpip_scores, marker='o', label=f"Guidance {scale}")

# --- GRAPH 1: SSIM ---
ax1.set_title("Structure Consistency (SSIM)\nHigher is Better ↑")
ax1.set_xlabel("Inference Steps")
ax1.set_ylabel("SSIM Score")
ax1.grid(True, linestyle='--', alpha=0.6)

# LEGEND OUTSIDE RIGHT
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# --- GRAPH 2: LPIPS ---
ax2.set_title("Style Similarity (LPIPS)\nLower is Better ↓")
ax2.set_xlabel("Inference Steps")
ax2.set_ylabel("LPIPS Distance")
ax2.grid(True, linestyle='--', alpha=0.6)

# LEGEND OUTSIDE RIGHT
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# Adjust layout to make room for legends
plt.tight_layout()

# Save with bbox_inches='tight' so the legend doesn't get cropped
plt.savefig("ablation_chart.png", bbox_inches='tight', dpi=150)
plt.show()

Overwriting analysis_ablation.py


In [ ]:
# @title Diffusion top 20 similar characters
%cd /content/
!python font_diffusion/sample_excel.py \
    --excel_file "SinoNom_Similar_Dic_v2.xlsx" \
    --style_image_path "font_diffusion/data_examples/sampling/example_style.jpg" \
    --ckpt_dir "./ckpt" \
    --ttf_path "HanaMinA.otf" \
    --ttf_path "HanaMinB.otf" \
    --ttf_path "HanaMinC.otf" \
    --ttf_path "NomNaTong-Regular.ttf" \
    --output_base_dir "./font_generations" \
    --device "cuda:0" \
    --debug

/content
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html

FONTDIFFUSER BATCH PROCESSING

Loading fonts...
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default

Loading 4 font(s)...
  ✓ Loaded: Hanazono Mincho A Regular
  ✓ Loaded: Hanazono Mincho B Regular
  ✓ Load

In [ ]:
# @title Zipping the results folder
!zip -r /content/font_generations.zip /content/font_generations
print(f"Finish zipped the font_generations folder! Ready for downloading")

In [ ]:
# @title Happy Christmas✨
# !rm -r -f font_diffusion